In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Parâmetros
TAMANHO_POPULACAO = 100  # Tamanho da população inicial
TAXA_MUTACAO = 0.1      # Chance de mutação (10%)
GERACOES = 10           # Número de gerações
TAMANHO_TABULEIRO = 8   # Tamanho do tabuleiro e número de rainhas

# Função para criar um indivíduo
def criar_individuo():
    # Cada indivíduo é uma permutação de números de 0 a 7, representando
    # uma posição de rainha em cada coluna
    return np.random.permutation(TAMANHO_TABULEIRO)

# Função para calcular o fitness de um indivíduo
def calcular_fitness(individuo):
    # O fitness é o número de pares de rainhas que não se atacam
    pares_nao_atacantes = 0
    for i in range(TAMANHO_TABULEIRO):
        for j in range(i + 1, TAMANHO_TABULEIRO):
            # Verifica se as rainhas i e j não estão na mesma linha ou diagonal
            if abs(individuo[i] - individuo[j]) != j - i:
                pares_nao_atacantes += 1
    return pares_nao_atacantes

# Função para criar uma população inicial
def criar_populacao(tamanho):
    # Retorna uma lista de indivíduos aleatórios
    return [criar_individuo() for _ in range(tamanho)]

# Função de seleção usando um torneio
def selecionar(populacao):
    # Seleciona dois indivíduos com melhor fitness para serem os pais
    torneio = random.sample(populacao, 5)  # Seleciona 5 indivíduos aleatoriamente
    torneio.sort(key=lambda ind: -calcular_fitness(ind))  # Ordena pelo fitness
    return torneio[0], torneio[1]  # Retorna os dois melhores

# Função de crossover para combinar dois pais e criar um filho
def crossover(pai1, pai2):
    # Combinação aleatória de partes de cada pai
    ponto_corte = random.randint(1, TAMANHO_TABULEIRO - 1)
    filho = np.zeros(TAMANHO_TABULEIRO, dtype=int) - 1  # Inicializa o filho com -1
    filho[:ponto_corte] = pai1[:ponto_corte]  # Copia parte do primeiro pai

    # Completa o filho com genes do segundo pai sem duplicações
    for gene in pai2:
        if gene not in filho:
            idx = np.where(filho == -1)[0][0]
            filho[idx] = gene
    return filho

# Função de mutação para alterar aleatoriamente um indivíduo
def mutar(individuo):
    # Aplica a mutação trocando duas posições aleatórias com chance definida
    if random.random() < TAXA_MUTACAO:
        idx1, idx2 = random.sample(range(TAMANHO_TABULEIRO), 2)
        individuo[idx1], individuo[idx2] = individuo[idx2], individuo[idx1]

# Função principal do algoritmo genético
def algoritmo_genetico():
    populacao = criar_populacao(TAMANHO_POPULACAO)  # Cria população inicial
    melhor_fitness = 0  # Variável para armazenar o melhor fitness
    melhor_individuo = None  # Variável para armazenar o melhor indivíduo

    for geracao in range(GERACOES):
        # Ordena a população por fitness em ordem descendente
        populacao.sort(key=lambda ind: -calcular_fitness(ind))

        # Armazena o melhor indivíduo se ele for encontrado
        if calcular_fitness(populacao[0]) > melhor_fitness:
            melhor_fitness = calcular_fitness(populacao[0])
            melhor_individuo = populacao[0]

        # Verifica se encontrou a solução ideal
        if melhor_fitness == (TAMANHO_TABULEIRO * (TAMANHO_TABULEIRO - 1)) // 2:
            break

        # Gera uma nova população usando seleção, crossover e mutação
        nova_populacao = []
        for _ in range(TAMANHO_POPULACAO // 2):
            pai1, pai2 = selecionar(populacao)  # Seleciona pais
            filho1, filho2 = crossover(pai1, pai2), crossover(pai2, pai1)  # Crossover
            mutar(filho1)  # Aplica mutação ao filho 1
            mutar(filho2)  # Aplica mutação ao filho 2
            nova_populacao.extend([filho1, filho2])  # Adiciona filhos à nova população
        populacao = nova_populacao  # Atualiza a população para a próxima geração

    return melhor_individuo  # Retorna o melhor indivíduo encontrado

# Função para exibir o tabuleiro e a solução usando matplotlib
def exibir_solucao_com_matplotlib(solucao):
    fig, ax = plt.subplots(figsize=(6, 6))  # Configura o tamanho da figura

    # Colore o tabuleiro
    for linha in range(TAMANHO_TABULEIRO):
        for coluna in range(TAMANHO_TABULEIRO):
            cor = 'white' if (linha + coluna) % 2 == 0 else 'lightgray'
            ax.add_patch(plt.Rectangle((coluna, linha), 1, 1, color=cor))  # Adiciona o quadrado colorido

    # Posiciona as rainhas na solução usando a cor vermelha
    for coluna in range(TAMANHO_TABULEIRO):
        linha = solucao[coluna]
        ax.text(coluna + 0.5, linha + 0.5, '♕', ha='center', va='center', color='red', fontsize=24)

    # Configurações do gráfico para simular um tabuleiro de xadrez
    ax.set_xticks(np.arange(TAMANHO_TABULEIRO + 1))
    ax.set_yticks(np.arange(TAMANHO_TABULEIRO + 1))
    ax.grid(True, which='both', color='black', linewidth=1.5)  # Adiciona linhas de grade
    ax.set_xticklabels([])  # Remove labels dos eixos x
    ax.set_yticklabels([])  # Remove labels dos eixos y
    plt.xlim(0, TAMANHO_TABULEIRO)
    plt.ylim(0, TAMANHO_TABULEIRO)
    plt.gca().invert_yaxis()  # Inverte o eixo y para que a linha superior do tabuleiro seja a linha 0
    plt.show()  # Exibe o tabuleiro

# Executa o algoritmo genético e exibe a melhor solução encontrada
solucao = algoritmo_genetico()
print("Melhor solução encontrada (contagem de coluna em coluna):", solucao)
exibir_solucao_com_matplotlib(solucao)